## convert to prompt-based fly ash+slag

In [37]:
import pandas as pd
import json
import os
def format_value(value):
    if pd.isna(value):
        return "unknown"
    elif isinstance(value, (int, float)):
        return f"{value:.1f}"
    else:
        return str(value)


# System prompt which is commmon for all cases

In [38]:
system_prompt = (
        "You are an advanced alkali-activated concrete mix design analyst AI assistant, specializing in predicting the  compressive strength "
        " based on detailed mix composition, curing conditions, and testing parameters. Your goal is to "
        " estimate the expected compressive strength of the mixture given to you by the user."
    )
print(system_prompt)

You are an advanced alkali-activated concrete mix design analyst AI assistant, specializing in predicting the  compressive strength  based on detailed mix composition, curing conditions, and testing parameters. Your goal is to  estimate the expected compressive strength of the mixture given to you by the user.


# This is prompt for combined slag  + fly-ash based system 

In [39]:
def process_final_curing(final_curing_val):
    if type(final_curing_val) is str:
        if final_curing_val =="External exposure":
            return "Sample was left in the open air"
        elif final_curing_val == "Sealed outdoor":
            return "Sample was sealed in a container and left outside"
    elif isinstance(final_curing_val,(int, float)):
        return f"Sample was left in a controlled environment at {final_curing_val} degree Celsius"
    else:
        return "unknown"

In [40]:
def create_message_both(row):
    
    user_message = (
        "I am working with a specific alkali activated concrete mixture with the following properties. Please analyze the mixture and "
        "provide the estimated compressive strength based on its composition, curing conditions, and testing parameters.\n\n"
        "\n\nPrecursor is a mixture of slag and fly ash:\n"
        f"""chemical composition of slag (in mass percent): CaO {format_value(row['GGBFS_CaO'])}, SiO2 {format_value(row['GGBFS_SiO2'])}, 
        Al2O3 {format_value(row['GGBFS_Al2O3'])}, Fe2O3 {format_value(row['GGBFS_Fe2O3'])}, MgO {format_value(row['GGBFS_MgO'])},
        SO3 {format_value(row['GGBFS_SO3'])}, K2O {format_value(row['GGBFS_K2O'])}, Na2O {format_value(row['GGBFS_Na2O'])}, LOI {format_value(row['GGBFS_LOI'])}\n """       
        f"""chemical composition of fly ash (in mass percent): CaO {format_value(row['FA_CaO'])}, SiO2 {format_value(row['FA_SiO2'])},
        Al2O3 {format_value(row['FA_Al2O3'])}, Fe2O3 {format_value(row['FA_Fe2O3'])}, MgO {format_value(row['FA_MgO'])},
        SO3 {format_value(row['FA_SO3'])}, K2O {format_value(row['FA_K2O'])}, Na2O {format_value(row['FA_Na2O'])}, LOI {format_value(row['FA_LOI'])}\n """
       f"\n Fineness modulus of the binder is {format_value(row['Fine modulus (m2/kg)'])}(m^2/kg)\n"
        "\n\nAlkali activator is the combination of NaOH and Na2SiO3:\n"
        f" The Na2O content in the Na2SiO3 is {format_value(row['Na2O (l)'])} % per liter solution \n"
        f" The SiO2 content in the Na2SiO3 is {format_value(row['SiO2 (l)'])} % per liter solution \n"
        f" The concentration of NaOH is {format_value(row['Concentration (M) NaOH'])} % per liter solution \n"
        "\n\nThe mix design is as follows:\n"
        f"Fly Ash (FA) content: {format_value(row['FA (kg/m3)'])} kg/m^3\n"
        f"Ground Granulated Blast Furnace Slag (GGBFS) content: {format_value(row['GGBFS (kg/m3)'])} kg/m^3\n"
        f"Total aggregates content: {format_value(row['Total aggregates (kg in 1m3 mix)'])} kg/m^3\n"
        f"Water content (from all sources): {format_value(row['Total water (in solutions + additional) (kg in 1m3 mix)'])} kg/m^3\n"
        f"Total Na2SiO3 content: {format_value(row['Total Na2SiO3 (kg in 1m3 of mix)'])} kg/m^3\n"
        f"Total NaOH content: {format_value(row['Total NaOH (kg in 1m3 mix)'])} kg/m^3\n"
        f"super-plasticizer content: {format_value(row['Superplasticizer (kg in 1m3 mix)'])} kg/m^3\n"
        f"\n\nThe mixture underwent an initial curing process for {format_value(row['Initial curing time (day) '])} days at "
        f"{format_value(row['Initial curing temp (C)'])} degree Celsius.\n"
        f"\n After initial curing: {process_final_curing(row['Final curing temp (C)'])} till the day of testing"
        f"\n\n Compressive strength was conducted on a cylindrical specimen at the age of  {format_value(row['Test age'])} days."
    )

    assistant_message = (
        f"Given the above information provided the estimate of the  compressive strength for this alkali activated concrete mixture "
        f"is {format_value(row['Compressive Strength'])} MPa." 
    )

    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message},
        {"role": "assistant", "content": assistant_message}
    ]


# Example usage
row = {
    "GGBFS_CaO": 0.2,
    "GGBFS_SiO2": 0.3,
    "GGBFS_Al2O3": 0.1,
    "GGBFS_Fe2O3": 0.1,
    "GGBFS_MgO": 0.1,
    "GGBFS_SO3": 0.1,
    "GGBFS_K2O": 0.1,
    "GGBFS_Na2O": 0.1,
    "GGBFS_LOI": 0.1,
    "FA_CaO": 0.2,
    "FA_SiO2": 0.3,
    "FA_Al2O3": 0.1,
    "FA_Fe2O3": 0.1,
    "FA_MgO": 0.1,
    "FA_SO3": 0.1,
    "FA_K2O": 0.1,
    "FA_Na2O": 0.1,
    "FA_LOI": 0.1,
    "Na2O (l)": 0.1,
    "SiO2 (l)": 0.1,
    "Concentration (M) NaOH": 0.1,
    "Fine modulus (m2/kg)": 0.1,
    "FA (kg/m3)": 0.1,
    "GGBFS (kg/m3)": 0.1,
    "Total aggregates (kg in 1m3 mix)": 0.1,
    "Total water (in solutions + additional) (kg in 1m3 mix)": 0.1,
    "Total Na2SiO3 (kg in 1m3 of mix)": 0.1,
    "Total NaOH (kg in 1m3 mix)": 0.1,
    "Superplasticizer (kg in 1m3 mix)": 0.1,
    "Initial curing time (day) ": 10,
    "Initial curing temp (C)": 20,
    "Final curing temp (C)": 24,
    "Test age": 28,
    "Compressive Strength": 30
}
print("***system***")
print(create_message_both(row)[0]["content"])
print("***user***")
print(create_message_both(row)[1]["content"])
print("***assistant***")
print(create_message_both(row)[2]["content"])



***system***
You are an advanced alkali-activated concrete mix design analyst AI assistant, specializing in predicting the  compressive strength  based on detailed mix composition, curing conditions, and testing parameters. Your goal is to  estimate the expected compressive strength of the mixture given to you by the user.
***user***
I am working with a specific alkali activated concrete mixture with the following properties. Please analyze the mixture and provide the estimated compressive strength based on its composition, curing conditions, and testing parameters.



Precursor is a mixture of slag and fly ash:
chemical composition of slag (in mass percent): CaO 0.2, SiO2 0.3, 
        Al2O3 0.1, Fe2O3 0.1, MgO 0.1,
        SO3 0.1, K2O 0.1, Na2O 0.1, LOI 0.1
 chemical composition of fly ash (in mass percent): CaO 0.2, SiO2 0.3,
        Al2O3 0.1, Fe2O3 0.1, MgO 0.1,
        SO3 0.1, K2O 0.1, Na2O 0.1, LOI 0.1
 
 Fineness modulus of the binder is 0.1(m^2/kg)


Alkali activator is the 

## Only slag prompt conversion

In [41]:
def create_messages_slag(row):
    user_message = (
        "I am working with a specific alkali activated concrete mixture with the following properties. Please analyze the mixture and "
        "provide the estimated compressive strength based on its composition, curing conditions, and testing parameters.\n\n"
        "\n\nPrecursor is a mixture of slag and fly ash:\n"
        f"""chemical composition of slag (in mass percent): CaO {format_value(row['GGBFS_CaO'])}, SiO2 {format_value(row['GGBFS_SiO2'])}, 
        Al2O3 {format_value(row['GGBFS_Al2O3'])}, Fe2O3 {format_value(row['GGBFS_Fe2O3'])}, MgO {format_value(row['GGBFS_MgO'])},
        SO3 {format_value(row['GGBFS_SO3'])}, K2O {format_value(row['GGBFS_K2O'])}, Na2O {format_value(row['GGBFS_Na2O'])}, LOI {format_value(row['GGBFS_LOI'])}\n """       
       f"\n Fineness modulus of the binder is {format_value(row['Fine modulus (m2/kg)'])}(m^2/kg)\n"
        "\n\nAlkali activator is the combination of NaOH and Na2SiO3:\n"
        f" The Na2O content in the Na2SiO3 is {format_value(row['Na2O (l)'])} % per liter solution \n"
        f" The SiO2 content in the Na2SiO3 is {format_value(row['SiO2 (l)'])} % per liter solution \n"
        f" The concentration of NaOH is {format_value(row['Concentration (M) NaOH'])} % per liter solution \n"
        "\n\nThe mix design is as follows:\n"
        f"Ground Granulated Blast Furnace Slag (GGBFS) content: {format_value(row['GGBFS (kg/m3)'])} kg/m^3\n"
        f"Total aggregates content: {format_value(row['Total aggregates (kg in 1m3 mix)'])} kg/m^3\n"
        f"Water content (from all sources): {format_value(row['Total water (in solutions + additional) (kg in 1m3 mix)'])} kg/m^3\n"
        f"Total Na2SiO3 content: {format_value(row['Total Na2SiO3 (kg in 1m3 of mix)'])} kg/m^3\n"
        f"Total NaOH content: {format_value(row['Total NaOH (kg in 1m3 mix)'])} kg/m^3\n"
        f"super-plasticizer content: {format_value(row['Superplasticizer (kg in 1m3 mix)'])} kg/m^3\n"
        f"\n\nThe mixture underwent an initial curing process for {format_value(row['Initial curing time (day) '])} days at "
        f"{format_value(row['Initial curing temp (C)'])} degree Celsius.\n"
        f"\n After initial curing: {process_final_curing(row['Final curing temp (C)'])} till the day of testing"
        f"\n\n Compressive strength was conducted on a cylindrical specimen at the age of  {format_value(row['Test age'])} days."
    )
    assistant_message = (
        f"Given the above information provided the estimate of  compressive strength for this alkali activated concrete mixture "
        f"is {format_value(row['Compressive Strength'])} MPa."
    )

    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message},
        {"role": "assistant", "content": assistant_message}
    ]

# Example usage
row = {
    "GGBFS_CaO": 0.2,
    "GGBFS_SiO2": 0.3,
    "GGBFS_Al2O3": 0.1,
    "GGBFS_Fe2O3": 0.1,
    "GGBFS_MgO": 0.1,
    "GGBFS_SO3": 0.1,
    "GGBFS_K2O": 0.1,
    "GGBFS_Na2O": 0.1,
    "GGBFS_LOI": 0.1,
    "Na2O (l)": 0.1,
    "SiO2 (l)": 0.1,
    "Concentration (M) NaOH": 0.1,
    "Fine modulus (m2/kg)": 0.1,
    "GGBFS (kg/m3)": 0.1,
    "Total aggregates (kg in 1m3 mix)": 0.1,
    "Total water (in solutions + additional) (kg in 1m3 mix)": 0.1,
    "Total Na2SiO3 (kg in 1m3 of mix)": 0.1,
    "Total NaOH (kg in 1m3 mix)": 0.1,
    "Superplasticizer (kg in 1m3 mix)": 0.1,
    "Initial curing time (day) ": 10,
    "Initial curing temp (C)": 20,
    "Final curing temp (C)": 24,
    "Test age": 28,
    "Compressive Strength": 30
}
print("***system***")
print(create_messages_slag(row)[0]["content"])
print("***user***")
print(create_messages_slag(row)[1]["content"])
print("***assistant***")
print(create_messages_slag(row)[2]["content"])

***system***
You are an advanced alkali-activated concrete mix design analyst AI assistant, specializing in predicting the  compressive strength  based on detailed mix composition, curing conditions, and testing parameters. Your goal is to  estimate the expected compressive strength of the mixture given to you by the user.
***user***
I am working with a specific alkali activated concrete mixture with the following properties. Please analyze the mixture and provide the estimated compressive strength based on its composition, curing conditions, and testing parameters.



Precursor is a mixture of slag and fly ash:
chemical composition of slag (in mass percent): CaO 0.2, SiO2 0.3, 
        Al2O3 0.1, Fe2O3 0.1, MgO 0.1,
        SO3 0.1, K2O 0.1, Na2O 0.1, LOI 0.1
 
 Fineness modulus of the binder is 0.1(m^2/kg)


Alkali activator is the combination of NaOH and Na2SiO3:
 The Na2O content in the Na2SiO3 is 0.1 % per liter solution 
 The SiO2 content in the Na2SiO3 is 0.1 % per liter solutio

## Only Fly ash prompt generation

In [42]:
def create_messages_flyash(row):
    user_message = (
        "I am working with a specific alkali activated concrete mixture with the following properties. Please analyze the mixture and "
        "provide the estimated compressive strength based on its composition, curing conditions, and testing parameters.\n\n"
        "\n\nPrecursor is a mixture of slag and fly ash:\n"
        f"""chemical composition of fly ash (in mass percent): CaO {format_value(row['FA_CaO'])}, SiO2 {format_value(row['FA_SiO2'])},
        Al2O3 {format_value(row['FA_Al2O3'])}, Fe2O3 {format_value(row['FA_Fe2O3'])}, MgO {format_value(row['FA_MgO'])},
        SO3 {format_value(row['FA_SO3'])}, K2O {format_value(row['FA_K2O'])}, Na2O {format_value(row['FA_Na2O'])}, LOI {format_value(row['FA_LOI'])}\n """
       f"\n Fineness modulus of the binder is {format_value(row['Fine modulus (m2/kg)'])}(m^2/kg)\n"
        "\n\nAlkali activator is the combination of NaOH and Na2SiO3:\n"
        f" The Na2O content in the Na2SiO3 is {format_value(row['Na2O (l)'])} % per liter solution \n"
        f" The SiO2 content in the Na2SiO3 is {format_value(row['SiO2 (l)'])} % per liter solution \n"
        f" The concentration of NaOH is {format_value(row['Concentration (M) NaOH'])} % per liter solution \n"
        "\n\nThe mix design is as follows:\n"
        f"Fly Ash (FA) content: {format_value(row['FA (kg/m3)'])} kg/m^3\n"
        f"Total aggregates content: {format_value(row['Total aggregates (kg in 1m3 mix)'])} kg/m^3\n"
        f"Water content (from all sources): {format_value(row['Total water (in solutions + additional) (kg in 1m3 mix)'])} kg/m^3\n"
        f"Total Na2SiO3 content: {format_value(row['Total Na2SiO3 (kg in 1m3 of mix)'])} kg/m^3\n"
        f"Total NaOH content: {format_value(row['Total NaOH (kg in 1m3 mix)'])} kg/m^3\n"
        f"super-plasticizer content: {format_value(row['Superplasticizer (kg in 1m3 mix)'])} kg/m^3\n"
        f"\n\nThe mixture underwent an initial curing process for {format_value(row['Initial curing time (day) '])} days at "
        f"{format_value(row['Initial curing temp (C)'])} degree Celsius.\n"
        f"\n After initial curing: {process_final_curing(row['Final curing temp (C)'])} till the day of testing"
        f"\n\n Compressive strength was conducted on a cylindrical specimen at the age of  {format_value(row['Test age'])} days."
    )

    assistant_message = (
        f"Given the above information provided the estimate of the  compressive strength for this alkali activated concrete mixture "
        f"is {format_value(row['Compressive Strength'])} MPa." 
    )

    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message},
        {"role": "assistant", "content": assistant_message}
    ]


# Usage for "Only Fly Ash"
row = {
    "FA_CaO": 0.2,
    "FA_SiO2": 0.3,
    "FA_Al2O3": 0.1,
    "FA_Fe2O3": 0.1,
    "FA_MgO": 0.1,
    "FA_SO3": 0.1,
    "FA_K2O": 0.1,
    "FA_Na2O": 0.1,
    "FA_LOI": 0.1,
    "Na2O (l)": 0.1,
    "SiO2 (l)": 0.1,
    "Concentration (M) NaOH": 0.1,
    "Fine modulus (m2/kg)": 0.1,
    "FA (kg/m3)": 0.1,
    "Total aggregates (kg in 1m3 mix)": 0.1,
    "Total water (in solutions + additional) (kg in 1m3 mix)": 0.1,
    "Total Na2SiO3 (kg in 1m3 of mix)": 0.1,
    "Total NaOH (kg in 1m3 mix)": 0.1,
    "Superplasticizer (kg in 1m3 mix)": 0.1,
    "Initial curing time (day) ": 10,
    "Initial curing temp (C)": 20,
    "Final curing temp (C)": 24,
    "Test age": 28,
    "Compressive Strength": 30

}
print("***system***")
print(create_messages_flyash(row)[0]["content"])
print("***user***")
print(create_messages_flyash(row)[1]["content"])
print("***assistant***")
print(create_messages_flyash(row)[2]["content"])

***system***
You are an advanced alkali-activated concrete mix design analyst AI assistant, specializing in predicting the  compressive strength  based on detailed mix composition, curing conditions, and testing parameters. Your goal is to  estimate the expected compressive strength of the mixture given to you by the user.
***user***
I am working with a specific alkali activated concrete mixture with the following properties. Please analyze the mixture and provide the estimated compressive strength based on its composition, curing conditions, and testing parameters.



Precursor is a mixture of slag and fly ash:
chemical composition of fly ash (in mass percent): CaO 0.2, SiO2 0.3,
        Al2O3 0.1, Fe2O3 0.1, MgO 0.1,
        SO3 0.1, K2O 0.1, Na2O 0.1, LOI 0.1
 
 Fineness modulus of the binder is 0.1(m^2/kg)


Alkali activator is the combination of NaOH and Na2SiO3:
 The Na2O content in the Na2SiO3 is 0.1 % per liter solution 
 The SiO2 content in the Na2SiO3 is 0.1 % per liter solut

GPT search generated context

In [43]:
context_gpt_search = """
Alkali-activated concrete (AAC) is a sustainable alternative to traditional Portland cement concrete, offering comparable or superior mechanical properties. The compressive strength of AAC is influenced by various mix design parameters, including the type of precursor materials, activator composition, water-to-binder ratio, curing conditions, and the incorporation of supplementary materials. Below is a summary of how these parameters affect the compressive strength of AAC:

Precursor Materials:

Ground Granulated Blast Furnace Slag (GGBFS): Utilizing GGBFS as a precursor generally enhances compressive strength due to its high reactivity and calcium content. 
Springer Link
Fly Ash: Incorporating fly ash can improve workability but may reduce early-age strength. However, it contributes to long-term strength development. 
AIP Publishing
Metakaolin: The addition of metakaolin can increase early strength due to its high reactivity. 
Springer Link
Activator Composition:

Alkali Activator Type: Sodium silicate combined with sodium hydroxide is commonly used, with the ratio of SiO₂ to Na₂O (modulus) affecting strength. A modulus around 1.0 to 1.5 is often optimal. 
AIP Publishing
Activator Concentration: Higher concentrations of alkali activators can increase strength but may also lead to rapid setting and workability issues.
Water-to-Binder Ratio (w/b):

A lower w/b ratio typically enhances compressive strength by reducing porosity. However, excessively low w/b ratios can impair workability. 
AIP Publishing
Curing Conditions:

Temperature: Elevated curing temperatures can accelerate strength gain but may also induce thermal cracking.
Humidity: Maintaining adequate humidity during curing prevents moisture loss, which is crucial for strength development.
Supplementary Materials:

Aggregates: The type and grading of aggregates influence the strength and durability of AAC. Proper selection and proportioning are essential. 
Springer Link
Admixtures: Incorporating superplasticizers can improve workability without significantly affecting strength.

"""
print(context_gpt_search)


Alkali-activated concrete (AAC) is a sustainable alternative to traditional Portland cement concrete, offering comparable or superior mechanical properties. The compressive strength of AAC is influenced by various mix design parameters, including the type of precursor materials, activator composition, water-to-binder ratio, curing conditions, and the incorporation of supplementary materials. Below is a summary of how these parameters affect the compressive strength of AAC:

Precursor Materials:

Ground Granulated Blast Furnace Slag (GGBFS): Utilizing GGBFS as a precursor generally enhances compressive strength due to its high reactivity and calcium content. 
Springer Link
Fly Ash: Incorporating fly ash can improve workability but may reduce early-age strength. However, it contributes to long-term strength development. 
AIP Publishing
Metakaolin: The addition of metakaolin can increase early strength due to its high reactivity. 
Springer Link
Activator Composition:

Alkali Activator Ty

this context is generated by providing research paper to chatgpt to provide context 

In [44]:
context_gpt_research_paper = """
1. Activator Type:

Positive Impact:
Sodium silicate (Na₂SiO₃) combined with sodium hydroxide (NaOH) enhances compressive strength due to increased silica availability and reaction rates​(1-s2.0-S095006181631571…)​(1-s2.0-S095006182031385…).
Potassium-based activators (e.g., KOH) provide higher compressive strength compared to sodium-based activators because of higher basicity​(1-s2.0-S095006182031793…).
Negative Impact:
High activator concentration can cause excessive heat release, leading to microcracks and reduced strength​(1-s2.0-S095006182031793…)​(1-s2.0-S000888461500119…).
Low activator concentration results in incomplete reactions, reducing strength​(1-s2.0-S095006182031385…).
2. Precursor Composition:

Positive Impact:
High-calcium precursors (e.g., ground granulated blast furnace slag) promote faster strength gain through C-S-H gel formation​(1-s2.0-S000888461730687…)​(1-s2.0-S095006181631571…).
Blending slag with low-calcium precursors (e.g., Class F fly ash) provides balanced strength and durability​(1-s2.0-S095006182031385…)​(Letter160_12012023).
Negative Impact:
Pure low-calcium fly ash requires prolonged curing or elevated temperatures for adequate strength​(1-s2.0-S095006181631571…).
3. Curing Conditions:

Positive Impact:
Elevated temperature curing (60–80°C) accelerates reaction rates and enhances early compressive strength​(1-s2.0-S095006181631571…)​(1-s2.0-S095006182031793…).
Continuous moist curing prevents cracking and ensures complete reactions​(1-s2.0-S095006182031793…)​(Letter160_12012023).
Negative Impact:
Inadequate curing (e.g., insufficient moisture or low curing temperatures) reduces reaction efficiency and strength​(1-s2.0-S095006182031793…).
4. Water-to-Solid (W/S) Ratio:

Positive Impact:
Optimal W/S ratio minimizes porosity and improves microstructure density​(1-s2.0-S095006182031385…)​(1-s2.0-S095006182031793…).
Negative Impact:
Excessively high W/S ratio increases porosity, reducing strength​(Letter160_12012023).
Too low W/S ratio limits workability and results in incomplete reactions​(1-s2.0-S095006182031793…).
5. Additives and Admixtures:

Positive Impact:
Shrinkage-reducing admixtures and superplasticizers enhance workability and reduce cracking, indirectly supporting higher strength​(1-s2.0-S095006182031793…)​(1-s2.0-S000888461500119…).
Fibers mitigate shrinkage cracking, preserving compressive strength​(1-s2.0-S095006182031793…).
Negative Impact:
Incompatible admixtures may interfere with the alkali activation process​(Letter160_12012023).
6. Aggregate Type and Gradation:

Positive Impact:
Well-graded aggregates improve packing density and interfacial transition zones, boosting strength​(1-s2.0-S095006181631571…)​(1-s2.0-S095006182031793…).
Negative Impact:
Reactive aggregates may trigger alkali-silica reactions, reducing strength​(1-s2.0-S000888461500119…).
7. Setting Retarders and Reactivity Enhancers:

Positive Impact:
Setting retarders improve workability and prevent premature setting in high-calcium systems​(1-s2.0-S095006182031793…)​(Letter160_12012023).
Mechanochemical activation of precursors (e.g., ball milling) increases reactivity and improves strength​(1-s2.0-S095006182031385…)​(Letter160_12012023).
"""

print (context_gpt_research_paper)



1. Activator Type:

Positive Impact:
Sodium silicate (Na₂SiO₃) combined with sodium hydroxide (NaOH) enhances compressive strength due to increased silica availability and reaction rates​(1-s2.0-S095006181631571…)​(1-s2.0-S095006182031385…).
Potassium-based activators (e.g., KOH) provide higher compressive strength compared to sodium-based activators because of higher basicity​(1-s2.0-S095006182031793…).
Negative Impact:
High activator concentration can cause excessive heat release, leading to microcracks and reduced strength​(1-s2.0-S095006182031793…)​(1-s2.0-S000888461500119…).
Low activator concentration results in incomplete reactions, reducing strength​(1-s2.0-S095006182031385…).
2. Precursor Composition:

Positive Impact:
High-calcium precursors (e.g., ground granulated blast furnace slag) promote faster strength gain through C-S-H gel formation​(1-s2.0-S000888461730687…)​(1-s2.0-S095006181631571…).
Blending slag with low-calcium precursors (e.g., Class F fly ash) provides balan

Read and filter dataset

In [45]:
datapath = "data/processed_data_full_28days_strength.xlsx"
import pandas as pd

def filter_data(datapath, mix_type):
    df = pd.read_excel(datapath)
    if mix_type == "flyash":
        return df[(df["GGBFS (kg/m3)"] == 0) | (df["GGBFS (kg/m3)"].isnull())]
    elif mix_type == "ggbfs":
        return df[(df["FA (kg/m3)"] == 0) | (df["FA (kg/m3)"].isnull())]
    elif mix_type == "combined":
        return df[(df["GGBFS (kg/m3)"] != 0) & (~df["GGBFS (kg/m3)"].isnull()) & (df["FA (kg/m3)"] != 0) & (~df["FA (kg/m3)"].isnull())]
    else:
        raise ValueError("Invalid mix type. Choose from 'flyash', 'ggbfs', or 'combined'.")
    
# Example usage
dfflyash = filter_data(datapath, "flyash")
print(dfflyash.shape)
dfslag = filter_data(datapath, "ggbfs")
print(dfslag.shape)
dfboth = filter_data(datapath, "combined")
print(dfboth.shape)

(351, 58)
(125, 58)
(296, 58)


In [46]:
#example prompt generation with excel data both precursors 
dfboth = filter_data(datapath, "combined")
msg = create_message_both(dfboth.iloc[0])
print("***system***")
print(msg[0]["content"])
print("***user***")
print(msg[1]["content"])
print("***assistant***")
print(msg[2]["content"])

***system***
You are an advanced alkali-activated concrete mix design analyst AI assistant, specializing in predicting the  compressive strength  based on detailed mix composition, curing conditions, and testing parameters. Your goal is to  estimate the expected compressive strength of the mixture given to you by the user.
***user***
I am working with a specific alkali activated concrete mixture with the following properties. Please analyze the mixture and provide the estimated compressive strength based on its composition, curing conditions, and testing parameters.



Precursor is a mixture of slag and fly ash:
chemical composition of slag (in mass percent): CaO 6.3, SiO2 5.4, 
        Al2O3 1.8, Fe2O3 0.3, MgO 0.9,
        SO3 0.0, K2O 0.0, Na2O 0.0, LOI 0.0
 chemical composition of fly ash (in mass percent): CaO 2.6, SiO2 41.9,
        Al2O3 19.3, Fe2O3 13.6, MgO 0.9,
        SO3 0.8, K2O 0.0, Na2O 0.0, LOI 2.5
 
 Fineness modulus of the binder is 503.5(m^2/kg)


Alkali activator is

In [47]:
#example prompt generation with excel data slag
dfslag = filter_data(datapath, "ggbfs")
msg = create_messages_slag(dfslag.iloc[0])
print("***system***")
print(msg[0]["content"])
print("***user***")
print(msg[1]["content"])
print("***assistant***")
print(msg[2]["content"])

***system***
You are an advanced alkali-activated concrete mix design analyst AI assistant, specializing in predicting the  compressive strength  based on detailed mix composition, curing conditions, and testing parameters. Your goal is to  estimate the expected compressive strength of the mixture given to you by the user.
***user***
I am working with a specific alkali activated concrete mixture with the following properties. Please analyze the mixture and provide the estimated compressive strength based on its composition, curing conditions, and testing parameters.



Precursor is a mixture of slag and fly ash:
chemical composition of slag (in mass percent): CaO 41.7, SiO2 33.5, 
        Al2O3 13.5, Fe2O3 0.3, MgO 6.0,
        SO3 2.7, K2O 0.3, Na2O 0.2, LOI 0.0
 
 Fineness modulus of the binder is unknown(m^2/kg)


Alkali activator is the combination of NaOH and Na2SiO3:
 The Na2O content in the Na2SiO3 is 0.1 % per liter solution 
 The SiO2 content in the Na2SiO3 is 0.3 % per liter 

In [48]:
#example prompt generation with excel data flyash
dfflyash = filter_data(datapath, "flyash")
msg = create_messages_flyash(dfflyash.iloc[0])
print("***system***")
print(msg[0]["content"])
print("***user***")
print(msg[1]["content"])
print("***assistant***")
print(msg[2]["content"])


***system***
You are an advanced alkali-activated concrete mix design analyst AI assistant, specializing in predicting the  compressive strength  based on detailed mix composition, curing conditions, and testing parameters. Your goal is to  estimate the expected compressive strength of the mixture given to you by the user.
***user***
I am working with a specific alkali activated concrete mixture with the following properties. Please analyze the mixture and provide the estimated compressive strength based on its composition, curing conditions, and testing parameters.



Precursor is a mixture of slag and fly ash:
chemical composition of fly ash (in mass percent): CaO 2.4, SiO2 47.8,
        Al2O3 24.4, Fe2O3 17.4, MgO 2.0,
        SO3 0.3, K2O 0.6, Na2O 0.3, LOI 1.1
 
 Fineness modulus of the binder is unknown(m^2/kg)


Alkali activator is the combination of NaOH and Na2SiO3:
 The Na2O content in the Na2SiO3 is 0.1 % per liter solution 
 The SiO2 content in the Na2SiO3 is 0.3 % per lite